## Fraud Detection Notebook using Machine Learning

In [ ]:
#!pip install aix360

In [ ]:
#!pip install lightgbm

## Part-1 Protodash Explainer

* The method selects applications from the training set that are similar in different ways to the user application we want to explain. For example, a user is predicted at Fraud Risk due to which he/she is denied loan. This could be because may be rejected justifiably because credit score is too low were low similar to another user who is at fraud risk , or because his/her loan amounts were too high as compared to the Applicant's income similar to a different rejected user.


* It doesn't give standard explanation for every case by using basic similarity techniques such as which use metrics such as euclidean distance, cosine similarity amongst others. 
`Protodash provides a much more well rounded and comprehensive view of why the decision for the applicant may be justifiable.`


More Technical definition of Protodash : 

ProtodashExplainer provides exemplar-based explanations for summarizing datasets as well
as explaining predictions made by an AI model. It employs a fast gradient based algorithm to find prototypes along with their (non-negative) importance weights. The algorithm minimizes the maximummean discrepancy metric and has constant factor approximation guarantees for this weakly submodular function.

 [References:](https://arxiv.org/abs/1707.01212).
   Paper by : `Karthik S. Gurumoorthy, Amit Dhurandhar, Guillermo Cecchi,"ProtoDash: Fast Interpretable Prototype Selection"
        

In [1]:


import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model, load_model, model_from_json
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
import numpy as np

from aix360.algorithms.protodash import ProtodashExplainer

Using TensorFlow backend.


#### load the data again here! Follow the steps: 

* Load the `Fraud-Data` as csv in the notebook.
* Click on the 0100 on the top right corner.
* Drag and Drop Fraud-Data.csv
* Click on `Insert to Code` and then `Pandas Dataframe.`
* Name the dataframe as `df`


In [2]:

import types

from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6f36c7b669bf4bb58c56051ea1508b9b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Cw2eSp0GZkC_r4hg3l-sZCm_96xjGMJcro-qe7YTX4LN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6f36c7b669bf4bb58c56051ea1508b9b.get_object(Bucket='fraudpredictionampaiexplainablity-donotdelete-pr-m8jztpooxxxpcj',Key='fraud_dataset.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Term,Credit_History_Available,Housing,Locality,Fraud_Risk
0,1,0,0,1,0,5849,0,146,360,1,1,1,0
1,1,1,1,1,1,4583,1508,128,360,1,1,3,1
2,1,1,0,1,1,3000,0,66,360,1,1,1,1
3,1,1,0,0,1,2583,2358,120,360,1,1,1,1
4,1,0,0,1,0,6000,0,141,360,1,1,1,0


In [3]:
df.iloc[[2, 3, 5], [12]]

,Fraud_Risk
2,1
3,1
5,1


In [4]:
df.head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
Gender,1,1,1,1,1,1,1,1,1,1
Married,0,1,1,1,0,1,1,1,1,1
Dependents,0,1,0,0,0,2,0,3,2,1
Education,1,1,1,0,1,1,0,1,1,1
Self_Employed,0,1,1,1,0,1,1,1,1,1
ApplicantIncome,5849,4583,3000,2583,6000,5417,2333,3036,4006,12841
CoapplicantIncome,0,1508,0,2358,0,4196,1516,2504,1526,10968
LoanAmount,146,128,66,120,141,267,95,158,168,349
Loan_Term,360,360,360,360,360,360,360,360,360,360
Credit_History_Available,1,1,1,1,1,1,1,0,1,1


In [5]:
X = df[df.columns[0:12]]
y = df[df.columns[12:]]

### Splitting the data with 70:30 mix

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


In [7]:
# Idx = 0 -> fraud_risk = 0 
# Idx = 1 -> fraud_risk = 1
# Idx = 2 -> fraud_risk = 1
# Idx = 3 -> fraud_risk = 1
# Idx = 4 -> fraud_risk = 0

In [8]:
# X_test.transpose()

In [9]:
# idx= 2
# x_test = X_test.to_numpy()
# x_test[idx].reshape((1, ) + x_test[idx].shape)

In [10]:
# print(X_test.iloc[4])

# print(y_test.iloc[[4]])

In [11]:
Z = np.vstack((X_train, X_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)

#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:X_train.shape[0], :]
xn_test  = N[X_train.shape[0]:, :]



In [12]:
xn_test[4]

array([ 0.5       ,  0.5       , -0.16666667,  0.5       ,  0.5       ,
       -0.45959184, -0.45800034, -0.24384949,  0.24358974,  0.5       ,
       -0.5       , -0.5       ])

In [13]:
# model = Sequential()
# model.add(Dense(500, input_dim=2, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:

def nn_small():
    model = Sequential()
    model.add(Dense(10, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal'  )  )
    model.add(Dense(2, kernel_initializer='normal'  )  )
    return model 

In [40]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 


class_names = ['Fraud-Risk', 'No-Fraud-Risk']
# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

# compile and print model summary
nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()

es = EarlyStopping(monitor='val_loss', patience=10)

history = nn.fit(xn_train, y_train, batch_size=50, epochs=200, verbose=1, shuffle=False, callbacks=[es])


# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 44        
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 10        
Total params: 184
Trainable params: 184
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
578/578 [==============================] - 0s 202us/step - loss: 0.8131 - accuracy: 0.8529
Epoch 2/200
578/578 [==============================] - 0s 43us/step - loss: 0.8131 - accuracy: 0.4602
Epoch 3/200
578/578 [==============================] - 0s 55us/step - loss: 0.8131 - accuracy: 0.4221
Epoch 4/200
578/578 [==============================] - 0s 42us/step - loss: 0.8

In [16]:
p_train = nn.predict_classes(xn_train) # Use trained neural network to predict train points
# print(p_train)
p_train = p_train.reshape((p_train.shape[0],1))

z_train = np.hstack((xn_train, p_train)) # Store (normalized) instances that were predicted as Good/No Fraud Risk
z_train_good = z_train[z_train[:,-1]==1, :]

zun_train = np.hstack((X_train, p_train)) # Store (unnormalized) instances that were predicted as Good 
zun_train_good = zun_train[zun_train[:,-1]==1, :]
# print(zun_train_good)

In [17]:
p_test = nn.predict_classes(xn_test)

In [18]:
print(p_test)

[0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 1 1 0 0 1 0 1 0 0 0 1 1 1 1 1 1 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0
 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 1
 0 0 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 1 0 1 0
 0 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 0 0 1 1 1 0 0
 0 1 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0
 0 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0]


In [41]:
idx = 4

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
# print(X)
print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

# attach the prediction made by the model to X
X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))
print(X)

x_test = X_test.to_numpy()
Xun = x_test[idx].reshape((1, ) + x_test[idx].shape)
print(Xun)

dfx = pd.DataFrame.from_records(Xun) # Create dataframe with original feature values
dfx
dfx[12] = int(X[0, -1])
dfx.columns = df.columns
dfx.transpose()

Chosen Sample: 4
Prediction made by the model: No-Fraud-Risk
Prediction probabilities: [[5001.8613 5002.0195]]

[[ 0.5         0.5        -0.16666667  0.5         0.5        -0.45959184
  -0.45800034 -0.24384949  0.24358974  0.5        -0.5        -0.5
   1.        ]]
[[   1    1    1    1    1 3417 1750  186  360    1    0    1]]


,0
Gender,1
Married,1
Dependents,1
Education,1
Self_Employed,1
ApplicantIncome,3417
CoapplicantIncome,1750
LoanAmount,186
Loan_Term,360
Credit_History_Available,1


In [42]:
explainer = ProtodashExplainer()
(W, S, setValues) = explainer.explain(X, z_train_good, m=5)

In [43]:
class_names = ['Fraud-Risk ', 'No-Fraud-Risk']

In [44]:
dfs = pd.DataFrame.from_records(zun_train_good[S, 0:-1].astype('double'))
RP=[]
for i in range(S.shape[0]):
    RP.append(class_names[int(z_train_good[S[i], -1])]) # Append class names
dfs[23] = RP
dfs.columns = df.columns  
dfs["Weight"] = np.around(W, 5)/np.sum(np.around(W, 5)) # Calculate normalized importance weights
dfs.transpose()


,0,1,2,3,4
Gender,1,1,1,0,1
Married,1,1,1,1,1
Dependents,1,2,0,0,1
Education,1,1,0,1,1
Self_Employed,1,1,1,1,1
ApplicantIncome,8072,1299,1800,2330,1538
CoapplicantIncome,240,1086,2934,4486,1425
LoanAmount,253,17,93,100,30
Loan_Term,360,120,360,360,360
Credit_History_Available,1,1,0,1,1


### Note : The above table `This gives the profile of the instances similar to each other who have no fraud risk to the loan office.`

In [45]:
z_train_bad = z_train[z_train[:,-1]==0, :]
zun_train_bad = zun_train[zun_train[:,-1]==0, :]

In [46]:
idx = 0 #another user to try 2385

X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Chosen Sample:", idx)
print("Prediction made by the model:", class_names[np.argmax(nn.predict_proba(X))])
print("Prediction probabilities:", nn.predict_proba(X))
print("")

X = np.hstack((X, nn.predict_classes(X).reshape((1,1))))

x_test = X_test.to_numpy()
# move samples to a dataframe to display
Xun = x_test[idx].reshape((1,) + x_test[idx].shape)
dfx = pd.DataFrame.from_records(Xun.astype('double'))
dfx[23] = class_names[int(X[0, -1])]
dfx.columns = df.columns
dfx.transpose()

Chosen Sample: 0
Prediction made by the model: Fraud-Risk 
Prediction probabilities: [[-2948.9329 -2949.1003]]



,0
Gender,0
Married,0
Dependents,0
Education,1
Self_Employed,1
ApplicantIncome,15759
CoapplicantIncome,0
LoanAmount,55
Loan_Term,360
Credit_History_Available,1


## Part-2 : Demostrating `Contrastive Explanations Method (CEM) algorithm using AI Explainability 360` on Fraud Data 

### Contrastive Explanations Method (CEM) algorithm available in AI Explainability 360.

* `The algorithm outputs a contrastive explanation which consists of two parts: a) pertinent negatives (PNs) and b) pertinent positives (PPs). PNs identify a minimal set of features which if altered would change the classification of the original input.`

For example,
 
Compute contrastive explanations for a few applicants
Given the trained NN model to decide on loan approvals based on the `fraud risk` , let us first examine an applicant whose application was denied and what (minimal) changes to his/her application would lead to approval (i.e. finding pertinent negatives). We will then look at another applicant whose loan was approved and ascertain features that would minimally suffice in him/her still getting a positive outcome (i.e. finding pertinent positives).

Compute Pertinent Negatives (PN):
In order to compute pertinent negatives, the CEM explainer computes a user profile that is close to the original applicant but for whom the decision of fraud risk is different. The explainer alters a minimal set of features by a minimal (positive) amount. This will help the user whose loan application was initially rejected say, to ascertain how to get it accepted.




In [47]:
from sklearn.model_selection import train_test_split
import numpy as np

In [48]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_6f36c7b669bf4bb58c56051ea1508b9b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Cw2eSp0GZkC_r4hg3l-sZCm_96xjGMJcro-qe7YTX4LN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_6f36c7b669bf4bb58c56051ea1508b9b.get_object(Bucket='fraudpredictionampaiexplainablity-donotdelete-pr-m8jztpooxxxpcj',Key='fraud_dataset.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df= pd.read_csv(body)



In [49]:
print("Number of frAUD risk applicants:", np.sum(df['Fraud_Risk']== 1 ))
print("Number of non-fraud risk applicants:", np.sum(df['Fraud_Risk']== 0))
print("Sample Applicants:")
df.head(10).transpose()

Number of frAUD risk applicants: 477
Number of non-fraud risk applicants: 350
Sample Applicants:


,0,1,2,3,4,5,6,7,8,9
Gender,1,1,1,1,1,1,1,1,1,1
Married,0,1,1,1,0,1,1,1,1,1
Dependents,0,1,0,0,0,2,0,3,2,1
Education,1,1,1,0,1,1,0,1,1,1
Self_Employed,0,1,1,1,0,1,1,1,1,1
ApplicantIncome,5849,4583,3000,2583,6000,5417,2333,3036,4006,12841
CoapplicantIncome,0,1508,0,2358,0,4196,1516,2504,1526,10968
LoanAmount,146,128,66,120,141,267,95,158,168,349
Loan_Term,360,360,360,360,360,360,360,360,360,360
Credit_History_Available,1,1,1,1,1,1,1,0,1,1


In [50]:
X = df[df.columns[0:12]]
y = df[df.columns[12:]]

In [51]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [52]:
y

,Fraud_Risk
0,0
1,1
2,1
3,1
4,0
...,...
822,0
823,0
824,0
825,0


### Normalising the dataset 

In [53]:
Z = np.vstack((x_train, x_test))
Zmax = np.max(Z, axis=0)
Zmin = np.min(Z, axis=0)

#normalize an array of samples to range [-0.5, 0.5]
def normalize(V):
    VN = (V - Zmin)/(Zmax - Zmin)
    VN = VN - 0.5
    return(VN)
    
# rescale a sample to recover original values for normalized values. 
def rescale(X):
    return(np.multiply ( X + 0.5, (Zmax - Zmin) ) + Zmin)

N = normalize(Z)
xn_train = N[0:x_train.shape[0], :]
xn_test  = N[x_train.shape[0]:, :]

In [54]:
xn_test[0]

array([ 0.5       , -0.5       , -0.5       ,  0.5       , -0.5       ,
       -0.44103896, -0.5       , -0.32344428,  0.24358974, -0.5       ,
        0.5       ,  0.5       ])

### Define and train a Neural Network Classifier

In [55]:
# !pip install tensorflow-hub

In [56]:
from aix360.algorithms.contrastive import CEMExplainer, KerasClassifier

In [57]:
def nn_small():
    model = Sequential()
    model.add(Dense(10, input_dim=12, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, kernel_initializer='normal'))    
    return model   

In [58]:
# Set random seeds for repeatability
np.random.seed(1) 
tf.set_random_seed(2) 

class_names = ['fraud-risk', 'no-fraud-risk']

# loss function
def fn(correct, predicted):
    return tf.nn.softmax_cross_entropy_with_logits(labels=correct, logits=predicted)

# compile and print model summary
nn = nn_small()
nn.compile(loss=fn, optimizer='adam', metrics=['accuracy'])
nn.summary()

nn.fit(xn_train, y_train, batch_size=100, epochs=500, verbose=1, shuffle=False)


# evaluate model accuracy        
score = nn.evaluate(xn_train, y_train, verbose=0) #Compute training set accuracy
#print('Train loss:', score[0])
print('Train accuracy:', score[1])

score = nn.evaluate(xn_test, y_test, verbose=0) #Compute test set accuracy
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 10)                130       
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 22        
Total params: 152
Trainable params: 152
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
578/578 [==============================] - 0s 150us/step - loss: 0.8107 - accuracy: 0.2474
Epoch 2/500
578/578 [==============================] - 0s 17us/step - loss: 0.8107 - accuracy: 0.5087
Epoch 3/500
578/578 [==============================] - 0s 22us/step - loss: 0.8107 - accuracy: 0.6713
Epoch 4/500
578/578 [==============================] - 0s 22us/step - loss: 0.8107 - accuracy: 0.7889
Epoch 5/500
578/578 [==============================] - 0s 26us/step - loss: 0.8107 - accuracy: 0.8547
Epoch 6

In [59]:
# Some interesting user samples to try: 2344 449 1168 1272
idx =4
# print(xn_test[idx].reshape((1,) + xn_test[idx].shape))
      
X = xn_test[idx].reshape((1,) + xn_test[idx].shape)
print("Computing PN for Sample:", idx)
print("Prediction made by the model:", nn.predict_proba(X))
print("Prediction probabilities:", class_names[np.argmax(nn.predict_proba(X))])
print("")

mymodel = KerasClassifier(nn)
explainer = CEMExplainer(mymodel)

arg_mode = 'PN' # Find pertinent negatives
arg_max_iter = 1000 # Maximum number of iterations to search for the optimal PN for given parameter settings
arg_init_const = 10.0 # Initial coefficient value for main loss term that encourages class change
arg_b = 9 # No. of updates to the coefficient of the main loss term
arg_kappa = 0.2 # Minimum confidence gap between the PNs (changed) class probability and original class' probability
arg_beta = 1e-1 # Controls sparsity of the solution (L1 loss)
arg_gamma = 100 # Controls how much to adhere to a (optionally trained) auto-encoder
my_AE_model = None # Pointer to an auto-encoder
arg_alpha = 0.01 # Penalizes L2 norm of the solution
arg_threshold = 1. # Automatically turn off features <= arg_threshold if arg_threshold < 1
arg_offset = 0.5 # the model assumes classifier trained on data normalized
                # in [-arg_offset, arg_offset] range, where arg_offset is 0 or 0.5
# Find PN for applicant 1272
(adv_pn, delta_pn, info_pn) = explainer.explain_instance(X, arg_mode, my_AE_model, arg_kappa, arg_b,
                                                         arg_max_iter, arg_init_const, arg_beta, arg_gamma,
                                                            arg_alpha, arg_threshold, arg_offset)

Computing PN for Sample: 4
Prediction made by the model: [[-122.928635 -122.97294 ]]
Prediction probabilities: fraud-risk




Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

iter:0 const:[10.]
Loss_Overall:2.4430, Loss_Attack:2.4430
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:-122.9286, max_nontarget_lab_score:-122.9729

iter:500 const:[10.]
Loss_Overall:2.4430, Loss_Attack:2.4430
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:-122.9286, max_nontarget_lab_score:-122.9729

iter:0 const:[100.]
Loss_Overall:24.4304, Loss_Attack:24.4304
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:-122.9286, max_nontarget_lab_score:-122.9729

iter:500 const:[100.]
Loss_Overall:24.4304, Loss_Attack:24.4304
Loss_L2Dist:0.0000, Loss_L1Dist:0.0000, AE_loss:0.0
target_lab_score:-122.9286, max_nontarget_lab_score:-122.

In [63]:
Xpn = adv_pn
classes = [ class_names[np.argmax(nn.predict_proba(X))], class_names[np.argmax(nn.predict_proba(Xpn))], 'NIL' ]

print("Sample:", idx)
print("prediction(X)", nn.predict_proba(X), class_names[np.argmax(nn.predict_proba(X))])
print("prediction(Xpn)", nn.predict_proba(Xpn), class_names[np.argmax(nn.predict_proba(Xpn))] )


X_re = rescale(X) # Convert values back to original scale from normalized
Xpn_re = rescale(Xpn)
Xpn_re = np.around(Xpn_re.astype(np.double), 2)

delta_re = Xpn_re - X_re
delta_re = np.around(delta_re.astype(np.double), 2)
delta_re[np.absolute(delta_re) < 1e-4] = 0

X3 = np.vstack((X_re, Xpn_re, delta_re))

dfre = pd.DataFrame.from_records(X3) # Create dataframe to display original point, PN and difference (delta)
dfre[23] = classes

dfre.columns = df.columns
dfre.rename(index={0:'X',1:'X_PN', 2:'(X_PN - X)'}, inplace=True)
dfret = dfre.transpose()


def highlight_ce(s, col, ncols):
    if (type(s[col]) != str):
        if (abs(s[col]) > 1):
            return(['background-color: yellow']*ncols)    
    return(['background-color: white']*ncols)

dfret.style.apply(highlight_ce, col='X_PN', ncols=3, axis=1) 

Sample: 4
prediction(X) [[-122.928635 -122.97294 ]] fraud-risk
prediction(Xpn) [[-189.36261 -189.32542]] no-fraud-risk


,X,X_PN,(X_PN - X)
Gender,1.000000,0.000000,-1.000000
Married,0.000000,0.000000,0.000000
Dependents,0.000000,0.000000,0.000000
Education,1.000000,0.000000,-1.000000
Self_Employed,0.000000,0.000000,0.000000
ApplicantIncome,3185.000000,150.000000,-3035.000000
CoapplicantIncome,11174.000000,0.000000,-11174.000000
LoanAmount,153.000000,9.000000,-144.000000
Loan_Term,360.000000,12.000000,-348.000000
Credit_History_Available,1.000000,0.000000,-1.000000


## The above results show that the customer should have 'less loan Amount', 'Loan_Term' for it to classified as `No-Fraud-risk.`